# 목표

Conv based로 D 모델만 학습이 가능한지 확인.

1. 가상의 G 모델(CLSTMC, pretrained)로 testset에 대해 prediction들을 생성
2. 생성한 데이터를 npy 파일로 저장
3. D 모델에서 가짜와 진짜 데이터를 구분이 가능한지(예측 데이터와 실제 데이터에 실제로 차이가 있는지) 를 확인한다.

In [1]:
from multiprocessing import cpu_count
from pathlib import Path
from typing import List, Union, AnyStr
from math import pi
import pickle

import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn import model_selection
from tqdm import tqdm
import torch_optimizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import random
from torchvision.models.resnet import resnet18

import torch_burn as tb
import utils

In [2]:
tb.seed_everything(0)

In [3]:
EXPERIMENT_NAME = '1130-Train-G'
CHECKPOINT_DIR = Path('checkpoint', EXPERIMENT_NAME)
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

DS_X = Path('data/1130/X.npy')
DS_Y = Path('data/1130/Y.npy')
DS_PRED = Path('data/1130/pred.npy')

# 데이터셋 생성

In [4]:
def load_result(checkpoint_dir):
    d = Path(checkpoint_dir)
    X = np.load(d / 'result-X.npy')
    Y = np.load(d / 'result-Y.npy')
    P = np.load(d / 'result-P.npy')
    return X, Y, P

In [5]:
X, Y, P = load_result('checkpoint/1130-Scene3-CLSTMC4-X60')

In [6]:
X.shape, Y.shape, P.shape

((54167, 60, 6), (54167, 3), (54167, 3))

In [7]:
class DetectorDataset(Dataset):
    def __init__(self, X, Y, P, L):
        super(DetectorDataset, self).__init__()

        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        self.P = torch.tensor(P, dtype=torch.float32)
        self.L = L
        self.Xsize = self.X.shape[0] // self.L
        self.Psize = self.P.shape[0] // self.L
        self.dssize = self.Xsize + self.Psize

    def __len__(self):
        return self.dssize

    def __getitem__(self, idx):
        if idx < self.Xsize:
            input = self.X[idx:idx + self.L, -1, :3]
            target = torch.tensor([0], dtype=torch.float32)
        else:
            idx -= self.Xsize
            input = self.P[idx:idx + self.L]
            target = torch.tensor([1], dtype=torch.float32)
        return input, target

In [8]:
ds = DetectorDataset(X, Y, P, 60)

# 모델 생성

In [9]:
class ResBlock1d(nn.Module):
    expansion = 1
    
    def __init__(self, inchannels, channels, kernel_size, stride=1):
        super(ResBlock1d, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(inchannels, channels * self.expansion, kernel_size, stride=stride, padding=kernel_size // 2),
            nn.BatchNorm1d(channels),
            nn.LeakyReLU(),
            nn.Conv1d(channels, channels * self.expansion, kernel_size, stride=stride, padding=kernel_size // 2),
            nn.BatchNorm1d(channels)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(inchannels, channels * self.expansion, 1, stride=stride),
            nn.BatchNorm1d(channels)
        )
        self.act = nn.LeakyReLU()

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        identity = self.conv2(identity)
        x += identity
        x = self.act(x)

        return x

In [10]:
class ConvDetector(nn.Module):
    def __init__(self, block, layers):
        super(ConvDetector, self).__init__()
        
        self.inchannels = 64
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(3, self.inchannels, 7, stride=2, padding=3, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512 * block.expansion, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.transpose(1, 2)  # B, S, 6 --> B, 6, S
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.sigmoid(x)

        return x
    
    def _make_layer(self, block, channels, blocks, stride=1):
        layers = []
        layers.append(block(self.inchannels, channels, 3))
        self.inchannels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inchannels, channels, 3))
        
        return nn.Sequential(*layers)

In [11]:
model = ConvDetector(ResBlock1d, [2, 2, 2, 2]).cuda()

In [12]:
x, y = ds[0]

In [13]:
x.shape

torch.Size([60, 3])

In [14]:
y.shape

torch.Size([1])

In [15]:
model.eval()
with torch.no_grad():
    x, y = ds[0]
    print(model(x.unsqueeze(0).cuda()).cpu(), y)
model.train()
print()

tensor([[0.5095]]) tensor([0.])



In [16]:
criterion = nn.BCELoss().cuda()
optimizer = torch_optimizer.RAdam(model.parameters())

In [17]:
class AccuracyMetric(tb.metrics.Metric):
    def on_valid_epoch_begin(self, epoch: int):
        self.u = 0
        self.d = 0

    def on_train_epoch_begin(self, epoch: int):
        self.u = 0
        self.d = 0

    def get_value(self, outputs: torch.Tensor, targets: torch.Tensor, is_train: bool):
        outputs = outputs.detach().cpu().flatten()
        targets = targets.detach().cpu().flatten()
        outputs = (outputs > 0.5).type(torch.int)
        targets = (targets > 0.5).type(torch.int)
        self.u += (1 - torch.abs(outputs - targets)).sum()
        self.d += outputs.numel()
        return self.u / self.d

In [18]:
metrics = [tb.metrics.ModuleMetric(criterion, 'loss'),
           AccuracyMetric('acc')]

In [19]:
callbacks = [
    tb.callbacks.EarlyStopping(metrics[0]),
    tb.callbacks.LRDecaying(optimizer, metrics[0], patience=3),
    tb.callbacks.SaveCheckpoint({'model': model}, metrics[0], CHECKPOINT_DIR, 'best-ckpt.pth')
]

In [20]:
trainer = tb.Trainer(model, optimizer, metrics, callbacks, ncols=100)

In [21]:
trainer.fit(ds, train_valid_split=0.2, num_epochs=200, batch_size=256, shuffle=True, pin_memory=True)

[0002/0200] Train:   0%|                                                      | 0/6 [00:00<?, ?it/s]

Save checkpoint: val_loss decreased from inf to 0.6930488348007202


[0003/0200] Train:   0%|                                                      | 0/6 [00:00<?, ?it/s]

Save checkpoint: val_loss decreased from 0.6930488348007202 to 0.6925760805606842


[0004/0200] Train:   0%|                                                      | 0/6 [00:00<?, ?it/s]

...

val_loss is not improved for 8 epochs


[0074/0200] Train:   0%|                                                      | 0/6 [00:00<?, ?it/s]

Decaying lr from 9.765625e-07 to 4.8828125e-07 because val_loss did not improved for 3 epochs
val_loss is not improved for 9 epochs


[0074/0200] Validation: 100%|█████████| 2/2 [00:00<00:00,  4.38it/s, val_loss 0.0006 val_acc 1.0000]

val_loss is not improved for 10 epochs
Stop training because val_loss did not improved for 10 epochs
